In [ ]:
from bs4 import BeautifulSoup
import os
import requests
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import svm
from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_error
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD, Adam
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from keras.layers import Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import classification_report, accuracy_score
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd
import numpy as np
import xarray as xr
import seaborn as sns
from datetime import datetime, timedelta
from sklearn.ensemble import RandomForestRegressor
from pathlib import Path
import pandas as pd
import glob
from io import StringIO
import csv
from sys import argv

Step 1: Clustering should be applied for each ship

define a function that can cluster time THEN space for a ship filter by its id. Using extended DBSCAN, one can extract multiple routes for the same ship, and extract more specifiecd corelations between weather and speed of this ship.

In [ ]:
from geopy.distance import great_circle
def ST_DBSCAN(df, spatial_threshold, temporal_threshold, min_neighbors):
    cluster_label = 0
    NOISE = -1
    UNMARKED = 777777
    stack = []

    # initialize each point with unmarked
    df['cluster'] = UNMARKED
    
    # for each point in database
    for index, point in df.iterrows():
        if df.loc[index]['cluster'] == UNMARKED:
            neighborhood = retrieve_neighbors(index, df, spatial_threshold, temporal_threshold)
            
            if len(neighborhood) < min_neighbors:
                df.at[index, 'cluster'] = NOISE

            else: # found a core point
                cluster_label = cluster_label + 1
                df.at[index, 'cluster'] = cluster_label# assign a label to core point

                for neig_index in neighborhood: # assign core's label to its neighborhood
                    df.at[neig_index, 'cluster'] = cluster_label
                    stack.append(neig_index) # append neighborhood to stack
                
                while len(stack) > 0: # find new neighbors from core point neighborhood
                    current_point_index = stack.pop()
                    new_neighborhood = retrieve_neighbors(current_point_index, df, spatial_threshold, temporal_threshold)
                    
                    if len(new_neighborhood) >= min_neighbors: # current_point is a new core
                        for neig_index in new_neighborhood:
                            neig_cluster = df.loc[neig_index]['cluster']
                            if (neig_cluster != NOISE) & (neig_cluster == UNMARKED): 
                                # TODO: verify cluster average before add new point
                                df.at[neig_index, 'cluster'] = cluster_label
                                stack.append(neig_index)
    return df


def retrieve_neighbors(index_center, df, spatial_threshold, temporal_threshold):
    neigborhood = []

    center_point = df.loc[index_center]

    # filter by time 
    min_time = center_point['BaseDateTime'] - timedelta(minutes = temporal_threshold)
    max_time = center_point['BaseDateTime'] + timedelta(minutes = temporal_threshold)
    df = df[(df['BaseDateTime'] >= min_time) & (df['BaseDateTime'] <= max_time)]

    # filter by distance
    for index, point in df.iterrows():
        if index != index_center:
            distance = great_circle((center_point['LAT'], center_point['LON']), (point['LAT'], point['LON'])).meters
            if distance <= spatial_threshold:
                neigborhood.append(index)

    return neigborhood

def route_cluster(df, dtw_threshold):
    route_label = 0
    NOISE = -1
    UNMARKED = 777777
    stack = []

    # initialize each point with unmarked
    df['route'] = UNMARKED
    df = df.groupby('cluster').agg(lambda x: list(x))

        
    # for each trip cluster
    for index, point in df.iterrows():
        xxx.iloc[index][['COG','LAT','LON']].values
        if df.loc[index]['route'] == UNMARKED:
            route_label = route_label + 1
            df.at[index, 'route'] = route_label # assign a label to core point

            for neighbor_index, neighbor_point in df.iterrows():
                dtw_distance = dtw_ndim.distance(series1, series2)
    return df


Read a raw table of AIS data merged with weather information. "Raw" in sense of no subsampling is done so that all ship AIS-points are perserved.

In [ ]:
df = pd.read_csv('dna_raw19.csv').dropna()

Extract unique IDs 

In [ ]:
df.MMSI.value_counts()[20:30]

Select a ship with relatively enough points

In [ ]:
ship = df[df.MMSI == 367133160]
ship.BaseDateTime = pd.to_datetime(ship.BaseDateTime)

Apply clustering Algorithm --> Spatial Temporal DBSCAN by using threshold of 12 km and 4 hours and #min_neighbors is 3

In [ ]:
spatial_threshold = 12000 # meters
temporal_threshold = 240 # minutes
min_neighbors = 3
df_clustering = ST_DBSCAN(ship, spatial_threshold, temporal_threshold, min_neighbors)

Trying to Group by cluster then show only the ones with the longest distance

In [ ]:
pd.DataFrame(df_clustering.groupby('cluster').apply(lambda x : (great_circle((min(x['LAT']), min(x['LON'])), (max(x['LAT']), max(x['LON']))).meters, min(x['LAT']), max(x['LAT']),  min(x['LON']), max(x['LON']) )))

Clean the clusters using the mean of ckusters as a threshold

In [ ]:
counts = df_clustering.cluster.value_counts() > df_clustering.cluster.value_counts().mean()
colusters = counts[counts].index
clean_clusters = df_clustering[df_clustering.cluster.isin(colusters)]

Visualize either all clusters as lines or one cluster as scattered points with different colors which illustrates speed

In [ ]:

c = clean_clusters #[clean_clusters.cluster == 13]

fig = px.line_geo(c, lat=c.LAT, lon=c.LON, hover_data=[c.BaseDateTime, c.COG, c.SOG], color=c.cluster)
# fig = px.scatter_geo(c, lat=c.LAT, lon=c.LON, hover_data=[c.BaseDateTime, c.COG], color=c.SOG)

fig.update_geos(projection_type="natural earth")
fig.update_layout( autosize=False, template='plotly_dark',
        width=1200,
        height=900,
        hovermode="closest"   
        )

fig.show()

Step 2 Extracting similar routes using Dynamic-Time-Warping (DTW): 

In [ ]:
# all_c = c.groupby('cluster').agg(lambda x: list(x)).reset_index(drop=True)
# dtw_i = dtw.distance_matrix(all_c['LAT'].values) + dtw.distance_matrix(all_c['LON'].values) 

In [ ]:
# import numpy as np
# import seaborn as sns
# import matplotlib.pylab as plt
# plt.Figure(figsize=(20,2)) 
# ax = sns.heatmap(dtw_i, linewidth=0.5)
# plt.show()

Step 3: apply time serise analysis and predict the speed of the ship using the last couple AIS data points as input
This can be achived by e.g.  LSTM  Model